In [1]:
import pandas as pd
from ch2.stoats.calculate.cda import *
from ch2.data import *
from ch2.squeal.database import connect

_, log, db = connect(['-v 0'])
with db.session_context() as s:
    df = accumulate_cda_crr(64 + 12, 
                            expand_bookmarks(log, s,
                                             filter_bookmarks(s, 0,
                                                              read_bookmarks(s, '60s/0ms'))))
df.describe()

,CdA,Crr
count,9976,9976
unique,9976,9976
top,"(0, 2.507944495045925)","(96.40391047630885, 0)"
freq,1,1


In [7]:
from bokeh.plotting import figure, output_notebook, show

output_notebook()
f = figure(plot_width=500, plot_height=400)
f.multi_line(xs='CdA', ys='Crr', source=df, line_alpha=0.1, line_color='black')
f.xaxis.axis_label = 'CdA'
f.yaxis.axis_label = 'Crr'
show(f)

Loading BokehJS ...

In [8]:
import numpy as np

r = np.linspace(0, 1.5, 100)
w = r[1] - r[0]
cda_0 = pd.DataFrame(columns=['CdA'], data=[cda[1] for cda in df['CdA']])
bins = cda_0.groupby(pd.cut(cda_0['CdA'], r)).size()
print(bins.describe())

f = figure(plot_width=900, plot_height=300)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
show(f)

count    99.000000
mean     41.545455
std      20.067373
min      10.000000
25%      26.000000
50%      37.000000
75%      52.000000
max      98.000000
dtype: float64


In [12]:
import scipy as sp
from scipy.optimize import fmin

cda = pd.DataFrame({'CdA': 0.5 * (r[:-1] + r[1:]), 'n': bins.values})
f = figure(plot_width=900, plot_height=300)
# f.circle(x='CdA', y='n', source=cda)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
for order in range(2, 20, 2):
    coeff = sp.polyfit(cda['CdA'], cda['n'], order)
    p = sp.poly1d(coeff)
    print(order, fmin(lambda x: -p(x), 0.6, disp=0)[0])
    f.line(x=cda['CdA'], y=p(cda['CdA']), line_color='orange')
show(f)         


2 0.7344140625000003
4 0.6134179687499999
6 0.5499023437499997
8 0.5460937499999998
10 0.5404101562499999
12 0.5283984374999998
14 0.5197851562499998
16 0.5158593749999998
18 0.5221874999999998


In [5]:
with db.session_context() as s:
    df = accumulate_crr(67 + 12, 
                        expand_bookmarks(log, s,
                                         filter_bookmarks(s, 0,
                                                          read_bookmarks(s, '60s'))),
                       cda=0.55)
df.describe()

,Crr
count,9903.000000
mean,22.653153
std,43.914063
min,-276.931368
25%,0.141276
50%,28.757651
75%,50.003623
max,258.029486


In [6]:
r = np.linspace(0, 100, 100)
w = r[1] - r[0]
bins = df.groupby(pd.cut(df['Crr'], r)).size()
print(bins.describe())

f = figure(plot_width=900, plot_height=300)
f.quad(top=bins, left=r[:-1]+0.1*w, right=r[1:]-0.1*w, bottom=0)
show(f)



count     99.000000
mean      72.888889
std       46.220388
min        3.000000
25%       26.000000
50%       84.000000
75%      109.500000
max      178.000000
dtype: float64
